# Introduction

Welcome to this tutorial on using jupyter notebooks with Moveit 2. A great benefit of being able to interact with MoveIt via a Python notebook is the ability to rapidly prototype code. We hope you find this interface intuitive and that you gain value from using MoveIt via Python notebooks.

In this tutorial we will cover the following: 

* The required imports to run the notebook
* A motion planning example
* A teleoperation example

If you have suggestions or feedback for this tutorial please post an issue on GitHub (https://github.com/ros-planning/moveit2_tutorials) and tag @peterdavidfagan.

## Imports

Note: to launch this notebook and the nodes it depends on you must first specify a launch file. Details are provided earlier in this tutorial ().

In [1]:
import os
import time

import rclpy
from rclpy.duration import Duration
from rclpy.node import Node

from moveit.core.robot_state import RobotState
from moveit.planning import MoveItPy

from moveit_msgs.srv import ServoCommandType
from geometry_msgs.msg import PoseStamped
from std_srvs.srv import SetBool

from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory

import numpy as np

class PoseTracker(Node):
    """A node for tracking multiple pose trajectories."""

    def __init__(self):
        super().__init__("pose_tracker")
        self.logger = self.get_logger()
        
        # initialize target poses list
        self.poses = []

        # initialize motion planning client
        # set params
        robot_ip = "192.168.106.99"
        use_gripper = "true" 
        use_fake_hardware = "true" 

        moveit_config = (
            MoveItConfigsBuilder(robot_name="panda", package_name="franka_robotiq_moveit_config")
            .robot_description(file_path=get_package_share_directory("franka_robotiq_description") + "/urdf/robot.urdf.xacro",
                mappings={
                    "robot_ip": robot_ip,
                    "robotiq_gripper": use_gripper,
                    "use_fake_hardware": use_fake_hardware,
                    })
            .robot_description_semantic("config/panda.srdf.xacro", 
                mappings={
                    "robotiq_gripper": use_gripper,
                    })
            .trajectory_execution("config/moveit_controllers.yaml")
            .moveit_cpp(
                file_path=get_package_share_directory("panda_motion_planning_demos")
                + "/config/moveit_cpp.yaml"
            )
            .to_moveit_configs()
            ).to_dict()
        
        self.robot = MoveItPy(config_dict=moveit_config)
        self.arm = self.robot.get_planning_component("panda_arm")

        # initialize servo service client
        self.servo_command_client = self.create_client(ServoCommandType, "/servo_node/switch_command_type")
        while not self.servo_command_client.wait_for_service(timeout_sec=1.0):
            self.get_logger().info("service not available, waiting again...")
        self.servo_pause_client = self.create_client(SetBool, "/servo_node/pause_servo")
        while not self.servo_pause_client.wait_for_service(timeout_sec=1.0):
            self.get_logger().info("service not available, waiting again...")

        #initialize servo target pose publisher
        self.publisher = self.create_publisher(PoseStamped, "/servo_node/pose_target_cmds", 10)

    def _set_target_pose(self, pose):
        """Sets the target pose."""
        self.target_pose = pose

    def _get_current_pose(self):
        """Gets the current pose of the robot."""
        self.arm.set_start_state_to_current_state()
        robot_state = self.arm.get_start_state()
        pose = robot_state.get_pose("panda_link8") # TODO: change to end effector link
        return pose

    # TODO: properly implement this
    def _check_pose_threshold(self, pose):
        """Checks if the robot is within the threshold of the target pose."""
        # TODO: check orientation + read thresholds from config
        current_pose = self._get_current_pose()
        del_x = current_pose.position.x - pose.pose.position.x
        del_y = current_pose.position.y - pose.pose.position.y
        del_z = current_pose.position.z - pose.pose.position.z
        if np.linalg.norm([del_x, del_y, del_z]) < 0.1:
            return True

    def set_servo_command_type(self, command_type):
        """Sets the servo command type."""
        request = ServoCommandType.Request()
        request.command_type = command_type
        self.future = self.servo_command_client.call_async(request)
        rclpy.spin_until_future_complete(self, self.future)
        if self.future.result():
            self.get_logger().info("Servo command type set to: %s" % command_type)
        else:
            self.get_logger().error("Failed to set servo command type: %s" % command_type)

    def return_to_start(self):
        """Returns the robot to the start position."""
        # pause servo    
        request = SetBool.Request()
        request.data = True
        future = self.servo_pause_client.call_async(request)
        rclpy.spin_until_future_complete(self, future)
        self.logger.info("servo paused")

        # plan to ready config
        self.arm.set_start_state_to_current_state()
        self.arm.set_goal_state(configuration_name="home")
        plan = self.arm.plan()
        if plan:
            robot_trajectory = plan.trajectory
            self.robot.execute(robot_trajectory, controllers=[])

        # restart servo
        request = SetBool.Request()
        request.data = False
        future = self.servo_pause_client.call_async(request)
        rclpy.spin_until_future_complete(self, future)
        self.logger.info("servo started")

    def generate_waypoints(self, trajectory, num_points = 100):
        """Generates waypoints from a trajectory."""

        # trajectory generator
        positions, orientations = trajectory.sample_points(num_points=num_points)

        self.poses = []
        for (position, orientation) in zip(positions, orientations):
            pose = PoseStamped()
            pose.header.frame_id = "panda_link0"
            pose.pose.position = position
            pose.pose.orientation = orientation
            self.poses.append(pose)


    def track_pose_targets(self):
        """Sets the target pose."""

        # check if poses have been set
        if len(self.poses) == 0:
            self.get_logger().error("No poses set.")
            return

        while True:
            # set target pose
            self._set_target_pose(self.poses[0])

            if self._check_pose_threshold(self.target_pose):
                self.get_logger().info("Reached target pose: %s" % self.target_pose)

                # pop last pose from list
                self.poses.pop(0)

                # check if last pose
                if len(self.poses) == 0:
                    self.get_logger().info("Finished tracking poses.")
                    return

                # set new target pose
                self._set_target_pose(self.poses[0])

            # publish target pose
            self.target_pose.header.stamp = self.get_clock().now().to_msg()
            self.publisher.publish(self.target_pose)


## Setup

In [2]:
rclpy.init()
node = PoseTracker()

from geometry_msgs.msg import PoseStamped, Point, Quaternion

pose = PoseStamped()
pose.header.frame_id = "panda_link0"
position = Point()
position.x = 0.2
position.y = 0.5
position.z = 0.7

quaternion = Quaternion()
quaternion.w = 1

pose.pose.position = position
pose.pose.orientation = quaternion

node.set_servo_command_type(2)
node.publisher.publish(pose)

[INFO] [1707907661.556432799] [moveit_276515853.moveit_cpp_initializer]: Initialize rclcpp
[INFO] [1707907661.556487469] [moveit_276515853.moveit_cpp_initializer]: Initialize node parameters
[INFO] [1707907661.556501529] [moveit_276515853.moveit_cpp_initializer]: Initialize node and executor
[INFO] [1707907661.576371499] [moveit_276515853.moveit_cpp_initializer]: Spin separate thread
[INFO] [1707907661.582095258] [moveit_276515853.RDFLoader]: Loaded robot model in 0.00532696 seconds
[INFO] [1707907661.582281589] [moveit_276515853.robot_model]: Loading robot model 'panda'...
[INFO] [1707907661.628914876] [moveit_276515853.kdl_kinematics_plugin]: Joint weights for group 'panda_arm': 1 1 1 1 1 1 1
[INFO] [1707907661.748510681] [moveit_276515853.planning_scene_monitor]: Publishing maintained planning scene on 'monitored_planning_scene'
[INFO] [1707907661.748920334] [moveit_276515853.moveit_cpp]: Listening to '/joint_states' for joint states
[INFO] [1707907661.749688038] [moveit_276515853.c